<a href="https://colab.research.google.com/github/jeanlamarre16/datascience/blob/main/FouilleDonnee/3-ANN/TP3_ANN_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importation des Librairies

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim

from google.colab import drive

In [13]:
# Montage du dossier google drive pour la récupération des fichiers de données
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


## 1. Préparation des données

1.1 Importer les données Train et Test

In [17]:
# Chargement des données du fichier de data test.csv pour le dataframe train_clean
df_train_clean = pd.read_csv('/gdrive/MyDrive/Data/train_clean.csv')

# Chargement des données du fichier de data test.csv pour le dataframe test_clean
df_test_clean = pd.read_csv('/gdrive/MyDrive/Data/test_clean.csv')

In [19]:
%load_ext google.colab.data_table # Pour l'affichage des données formatées sous forme de table

In [22]:
df_test_clean.head()
#df_train_clean.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,IsAlone,Title
0,892,3,1,2,0,2,1,1
1,893,3,0,2,0,0,0,3
2,894,2,1,3,1,2,1,1
3,895,3,1,1,1,0,1,1
4,896,3,0,1,1,0,0,3


1.2 Depuis le Dataframe train, charger les features d'apprentissage dans un array numpy X_alltrain, et les labels (données à prévoir) dans un array numpy y_alltrain

In [23]:
X_alltrain = df_train_clean.drop(columns=['PassengerId']).to_numpy() # Features
y_alltrain = df_train_clean['Survived'].to_numpy() # Labels

1.3 Séparer les features et les labels en deux parties (train et dev), en attribuant 10% des exemples aux données de dev. afficher les nombres de lignes et de colonnes pour les 4 arrays.

In [54]:
from sklearn.model_selection import train_test_split

X_train, X_dev, y_train, y_dev = train_test_split(X_alltrain, y_alltrain, test_size=0.1, random_state=42)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_dev = scaler.transform(X_dev)

print("[feature x_train] : nombre de lignes et de colonnes  => FEATURES ", X_train.shape)
print("[labels y_train]  : nombre de lignes                 => LABELS", y_train.shape)
print("[feature x_dev]   : nombre de lignes et de colonnes  => FEATURES ", X_dev.shape)
print("[labels y_dev]    : nombre de lignes                 => LABELS ", y_dev.shape)

[feature x_train] : nombre de lignes et de colonnes  => FEATURES  (801, 8)
[labels y_train]  : nombre de lignes                 => LABELS (801,)
[feature x_dev]   : nombre de lignes et de colonnes  => FEATURES  (90, 8)
[labels y_dev]    : nombre de lignes                 => LABELS  (90,)


1.4 Afficher les 10 premières lignes de features et les 10 premiers labels.

In [55]:
print("[FEATURES] (10 premières lignes):\n", X_train[:10])
print("\n[LABELS] (10 premiers labels):\n", y_train[:10])

[FEATURES] (10 premières lignes):
 [[ 1.27186755  0.82096936  0.72706668 -1.59025593  0.44576419 -0.55643466
  -1.2356431   2.21316687]
 [-0.78624539  0.82096936 -1.37538967 -1.59025593  1.34515239 -0.55643466
  -1.2356431   0.2704308 ]
 [-0.78624539 -1.59222865  0.72706668  1.97446888  1.34515239 -0.55643466
   0.80929517 -0.70093723]
 [-0.78624539  0.82096936  0.72706668  0.78622728 -1.35301221  2.6271665
   0.80929517 -0.70093723]
 [-0.78624539  0.82096936  0.72706668 -1.59025593 -1.35301221 -0.55643466
   0.80929517 -0.70093723]
 [-0.78624539  0.82096936  0.72706668 -0.40201432 -1.35301221 -0.55643466
   0.80929517 -0.70093723]
 [ 1.27186755 -1.59222865  0.72706668  0.78622728  1.34515239 -0.55643466
  -1.2356431  -0.70093723]
 [ 1.27186755  0.82096936  0.72706668  0.78622728 -0.45362401 -0.55643466
   0.80929517 -0.70093723]
 [ 1.27186755 -0.38562964 -1.37538967  0.78622728 -0.45362401 -0.55643466
   0.80929517  1.24179884]
 [-0.78624539  0.82096936  0.72706668 -0.40201432 -1.3530

## 2. Définition du réseau de neurone

2.1 Définir et instancier une classe Titanic Model avec les caractéristiques suivantes :
- deux  couches cachées de 50 neurones.
- deux classes en sortie : Survivant ou non
- Des fonctions d'activation RELU
- un dropout paramétrable pour les 2 couches cachées

In [56]:
import torch.nn as nn
import torch.nn.functional as F

class TitanicModel(nn.Module):
    def __init__(self, input_size, keep_prob=0.5):
        super(TitanicModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 2)
        self.dropout = nn.Dropout(p=1 - keep_prob)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

2.2 : Définir des paramètres de nombre d'epochs (50) et de learning_rate (0.01)

In [57]:
nb_epochs = 50
learning_rate = 0.01

2.3 définir la taille du minibatch à 50. En déduire le nombre de boucle pour chaque epoch.

In [58]:
batch_size = 50
num_batches = len(X_train)

2.4 Définir un fonction de coût de type CrossEntropy

In [59]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

2.5. Définir un optimizer de type Adam, sans oublier le learning rate

In [60]:
model = TitanicModel(input_size=X_train.shape[1])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## 3. Apprentissage

3.1 exécuter l'apprentissage du modèle.
- Créer une boucle sur les epochs, qui contient elel-même une boucle sur les minibatchs.
- A chaque nouvelle itération sur les epochs, mélanger les données avec la méthode shuffle.
- Tous les 5 epochs afficher la valeur de la fonction de cout

In [61]:
from sklearn.utils import shuffle

for epoch in range(nb_epochs):
    X_train, y_train = shuffle(X_train, y_train)
    for i in range(num_batches):
        start = i * batch_size
        end = start + batch_size
        inputs = torch.tensor(X_train[start:end], dtype=torch.float32)
        targets = torch.tensor(y_train[start:end], dtype=torch.long)

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{nb_epochs}], Loss: {loss.item():.4f}')

Epoch [5/50], Loss: nan
Epoch [10/50], Loss: nan
Epoch [15/50], Loss: nan
Epoch [20/50], Loss: nan
Epoch [25/50], Loss: nan
Epoch [30/50], Loss: nan
Epoch [35/50], Loss: nan
Epoch [40/50], Loss: nan
Epoch [45/50], Loss: nan
Epoch [50/50], Loss: nan


3.2 Calculer la précision de la prévision sur les données dev

In [66]:
with torch.no_grad():
    model = TitanicModel(input_size=X_dev.shape[1])
    inputs = torch.tensor(X_dev, dtype=torch.float32)
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == torch.tensor(y_dev)).sum().item() / len(y_dev)
    print(f'Accuracy on dev set: {accuracy:.4f}')

Accuracy on dev set: 0.5222


3.3 Calculer prévisions sur les données de tests

In [68]:
with torch.no_grad():
    input_size = df_test_clean.drop(columns=['PassengerId']).shape[1]  # Obtenir la dimension d'entrée
    model = TitanicModel(input_size=input_size)  # Initialiser le modèle avec la dimension correcte
    inputs = torch.tensor(df_test_clean.drop(columns=['PassengerId']).values, dtype=torch.float32)
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)

# Création d'un DataFrame pour les résultats
df_results = pd.DataFrame({'PassengerId': df_test_clean['PassengerId'], 'Survived': predicted.numpy()})

3.4 Générer le fichier résultat et l'envoyer sur kaggle
Quel est votre score et votre classement ?

In [74]:
df_results.to_csv('file4Kaggle.csv', index=False)

Score obtenu en soumettant le fichier "file4Kaggle.csv" au site https://www.kaggle.com/competitions/titanic/submissions#
[Compétition : Titanic - Machine Learning from Disasterest]
**Score d'accuracy (taux de précision globale) : 0.59808**

**Classement (Leaderboard) : 13104**

3.5 (Optionnel) Exécuter une Cross Validationd ans une boucle pour trouver les meilleures valeurs de learning rate, de keep_prob et de nombre d'epochs.

Cross_Validation avec skkearn : https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators

In [73]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

def cross_validation(X, y, learning_rates, keep_probs, num_epochss):
  best_score = 0
  best_params = {}

  for lr in learning_rates:
    for kp in keep_probs:
      for ne in num_epochss:
        scores = []
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        for train_index, val_index in kf.split(X):
          X_train, X_val = X[train_index], X[val_index]
          y_train, y_val = y[train_index], y[val_index]

          model = TitanicModel(input_size=X_train.shape[1], keep_prob=kp)
          optimizer = optim.Adam(model.parameters(), lr=lr)

          for epoch in range(ne):
            with torch.no_grad():
              inputs = torch.tensor(X_val, dtype=torch.float32)
              outputs = model(inputs)
              _, predicted = torch.max(outputs, 1)
              accuracy = accuracy_score(y_val, predicted.numpy())
              scores.append(accuracy)

        avg_score = np.mean(scores)
        print(f"lr: {lr}, probabilité: {kp}, nombre époques: {ne}, Moyenne de précision: {avg_score:.4f}")

        if avg_score > best_score:
          best_score = avg_score
          best_params = {'taux apprentissage': lr, 'probabilité': kp, 'nombre époques': ne}

  print(f"paramètres: {best_params}, précision: {best_score:.4f}")
  return best_params

# Utilisation
learning_rates = [0.01, 0.001]
keep_probs = [0.5, 0.7]
num_epochss = [30, 50]

best_params = cross_validation(X_alltrain, y_alltrain, learning_rates, keep_probs, num_epochss)

lr: 0.01, probabilité: 0.5, nombre époques: 30, Moyenne de précision: 0.4476
lr: 0.01, probabilité: 0.5, nombre époques: 50, Moyenne de précision: 0.5536
lr: 0.01, probabilité: 0.7, nombre époques: 30, Moyenne de précision: 0.5351
lr: 0.01, probabilité: 0.7, nombre époques: 50, Moyenne de précision: 0.4951
lr: 0.001, probabilité: 0.5, nombre époques: 30, Moyenne de précision: 0.4831
lr: 0.001, probabilité: 0.5, nombre époques: 50, Moyenne de précision: 0.5097
lr: 0.001, probabilité: 0.7, nombre époques: 30, Moyenne de précision: 0.4438
lr: 0.001, probabilité: 0.7, nombre époques: 50, Moyenne de précision: 0.4693
paramètres: {'taux apprentissage': 0.01, 'probabilité': 0.5, 'nombre époques': 50}, précision: 0.5536


In [77]:
# Enregistrement du fichier file4Kaggle.csv dans mon drive pour un accès permanent
!cp file4Kaggle.csv /gdrive/MyDrive/Data/